<a href="https://colab.research.google.com/github/brendiuxx66/POO/blob/main/GestorCumplea%C3%B1os.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.2 MB/s eta 0:00:00


In [7]:
!pip install python-dotenv
from dotenv import load_dotenv

In [21]:
# Crear archivo .env en Colab
with open(".env", "w") as f:
    f.write("GMAIL_USER=brenda.ramirez0366@alumnos.udg.mx\n")
    f.write("GMAIL_PASSWORD=spqpdimhcnoepdfy\n")


In [34]:
%%writefile app.py
import csv
import os
import random
import smtplib
from datetime import datetime, date
from email.message import EmailMessage
from typing import List
import streamlit as st
from dotenv import load_dotenv

# Carga variables de entorno
load_dotenv()

# --- Clases ---

class Contacto:
    def __init__(self, nombre: str, fecha_nacimiento: str, email: str, mensaje_id: int = None):
        self.nombre = nombre
        self.email = email
        self.fecha_nacimiento = datetime.strptime(fecha_nacimiento, "%Y-%m-%d").date()
        self.mensaje_id = mensaje_id  # ID del mensaje asignado, puede ser None

    def dias_para_cumple(self) -> int:
        hoy = date.today()
        proximo_cumple = self.fecha_nacimiento.replace(year=hoy.year)
        if proximo_cumple < hoy:
            proximo_cumple = proximo_cumple.replace(year=hoy.year + 1)
        return (proximo_cumple - hoy).days

class GestorCumpleaños:
    def __init__(self, archivo_contactos="contactos.csv"):
        self.archivo_contactos = archivo_contactos
        self.contactos: List[Contacto] = self.cargar_contactos()

    def cargar_contactos(self) -> List[Contacto]:
        contactos = []
        if os.path.exists(self.archivo_contactos):
            with open(self.archivo_contactos, mode="r", newline="") as file:
                reader = csv.reader(file)
                for fila in reader:
                    if len(fila) >= 3:
                        nombre, fecha, email = fila[:3]
                        mensaje_id = int(fila[3]) if len(fila) > 3 and fila[3].isdigit() else None
                        contactos.append(Contacto(nombre, fecha, email, mensaje_id))
        return contactos

    def guardar_contactos(self):
        with open(self.archivo_contactos, mode="w", newline="") as file:
            writer = csv.writer(file)
            for c in self.contactos:
                fila = [
                    c.nombre,
                    c.fecha_nacimiento.isoformat(),
                    c.email,
                    c.mensaje_id if c.mensaje_id is not None else ""
                ]
                writer.writerow(fila)

    def agregar_contacto(self, contacto: Contacto):
        self.contactos.append(contacto)
        self.guardar_contactos()

    def actualizar_contacto(self, index: int, contacto: Contacto):
        self.contactos[index] = contacto
        self.guardar_contactos()

    def eliminar_contacto(self, index: int):
        del self.contactos[index]
        self.guardar_contactos()

class MensajeManager:
    def __init__(self, archivo_mensajes="mensajes.csv"):
        self.archivo_mensajes = archivo_mensajes
        self.mensajes: List[str] = self.cargar_mensajes()

    def cargar_mensajes(self) -> List[str]:
        mensajes = []
        if os.path.exists(self.archivo_mensajes):
            with open(self.archivo_mensajes, mode="r", newline="") as file:
                reader = csv.reader(file)
                mensajes = [fila[0] for fila in reader if fila]
        return mensajes

    def guardar_mensajes(self):
        with open(self.archivo_mensajes, mode="w", newline="") as file:
            writer = csv.writer(file)
            for m in self.mensajes:
                writer.writerow([m])

    def agregar_mensaje(self, mensaje: str):
        self.mensajes.append(mensaje)
        self.guardar_mensajes()

    def actualizar_mensaje(self, index: int, mensaje: str):
        self.mensajes[index] = mensaje
        self.guardar_mensajes()

    def eliminar_mensaje(self, index: int):
        del self.mensajes[index]
        self.guardar_mensajes()

    def obtener_mensaje_aleatorio(self, nombre: str) -> str:
        if not self.mensajes:
            return f"¡Feliz cumpleaños, {nombre}!"
        mensaje = random.choice(self.mensajes)
        return mensaje.replace("{nombre}", nombre)

class Correo:
    def __init__(self):
        self.email_origen = os.getenv("GMAIL_USER")
        self.contraseña = os.getenv("GMAIL_PASSWORD")

    def enviar_correo(self, destinatario: str, asunto: str, mensaje: str):
        if not self.email_origen or not self.contraseña:
            st.error("❌ Configura tu email y contraseña en el archivo .env")
            return

        msg = EmailMessage()
        msg.set_content(mensaje)
        msg["Subject"] = asunto
        msg["From"] = self.email_origen
        msg["To"] = destinatario

        try:
            with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
                smtp.login(self.email_origen, self.contraseña)
                smtp.send_message(msg)
                st.success(f"✅ Correo enviado a {destinatario}")
        except Exception as e:
            st.error(f"❌ Error al enviar correo: {e}")

# --- Función para enviar felicitaciones automáticas ---

def felicitar_cumpleaños_hoy(gestor: GestorCumpleaños, mensajes: MensajeManager, correo: Correo):
    hoy = date.today()
    enviados = 0
    for contacto in gestor.contactos:
        if contacto.fecha_nacimiento.month == hoy.month and contacto.fecha_nacimiento.day == hoy.day:
            if contacto.mensaje_id is not None and 0 <= contacto.mensaje_id < len(mensajes.mensajes):
                mensaje = mensajes.mensajes[contacto.mensaje_id].replace("{nombre}", contacto.nombre)
            else:
                mensaje = mensajes.obtener_mensaje_aleatorio(contacto.nombre)
            correo.enviar_correo(contacto.email, "¡Feliz cumpleaños!", mensaje)
            enviados += 1
    return enviados

# --- Interfaz Streamlit ---

def main():
    st.set_page_config(page_title="Gestor de Cumpleaños", page_icon="🎂")
    st.title("🎉 Gestor de Cumpleaños Automático")

    gestor = GestorCumpleaños()
    mensajes = MensajeManager()
    correo = Correo()

    # Enviar felicitaciones automáticamente al iniciar
    felicitar_cumpleaños_hoy(gestor, mensajes, correo)

    menu = st.sidebar.selectbox("Menú", [
        "Inicio",
        "Agregar Contacto",
        "Ver Cumpleaños",
        "Gestionar Mensajes"
    ])

    if menu == "Inicio":
        st.header("🏠 Inicio")
        st.write(f"📌 Contactos registrados: {len(gestor.contactos)}")
        st.write(f"💬 Mensajes personalizados: {len(mensajes.mensajes)}")

    elif menu == "Agregar Contacto":
        st.header("👤 Agregar Nuevo Contacto")
        nombre = st.text_input("Nombre completo")
        fecha_minima = date(1600, 1, 1)
        fecha_maxima = date.today()
        fecha = st.date_input("Fecha de nacimiento", min_value=fecha_minima, max_value=fecha_maxima, value=fecha_maxima)
        email = st.text_input("Correo electrónico")

        # Selector de mensaje para asignar
        opciones_mensajes = ["Ninguno"] + mensajes.mensajes
        mensaje_seleccionado = st.selectbox("Selecciona mensaje para este contacto (opcional):", opciones_mensajes)

        if st.button("Guardar Contacto"):
            if not nombre or not email:
                st.warning("⚠️ Nombre y email son obligatorios")
            elif "@" not in email:
                st.error("❌ Ingresa un email válido")
            else:
                mensaje_id = None
                if mensaje_seleccionado != "Ninguno":
                    mensaje_id = mensajes.mensajes.index(mensaje_seleccionado)
                nuevo_contacto = Contacto(nombre.strip(), fecha.strftime("%Y-%m-%d"), email.strip(), mensaje_id)
                gestor.agregar_contacto(nuevo_contacto)
                st.success("✅ Contacto guardado correctamente")

        # Editar o eliminar contactos
        if gestor.contactos:
            st.subheader("Editar o eliminar contactos")
            opciones = [f"{c.nombre} ({c.fecha_nacimiento.isoformat()})" for c in gestor.contactos]
            seleccionado = st.selectbox("Selecciona un contacto", opciones)
            idx = opciones.index(seleccionado)
            contacto_sel = gestor.contactos[idx]

            nombre_edit = st.text_input("Nombre", value=contacto_sel.nombre, key="nombre_edit")
            fecha_edit = st.date_input("Fecha de nacimiento", min_value=fecha_minima, max_value=fecha_maxima, value=contacto_sel.fecha_nacimiento, key="fecha_edit")
            email_edit = st.text_input("Correo electrónico", value=contacto_sel.email, key="email_edit")

            # Selector de mensaje para editar
            mensaje_actual = "Ninguno"
            if contacto_sel.mensaje_id is not None and 0 <= contacto_sel.mensaje_id < len(mensajes.mensajes):
                mensaje_actual = mensajes.mensajes[contacto_sel.mensaje_id]
            opciones_mensajes_edit = ["Ninguno"] + mensajes.mensajes
            mensaje_seleccionado_edit = st.selectbox("Mensaje asignado", opciones_mensajes_edit, index=opciones_mensajes_edit.index(mensaje_actual))

            if st.button("Actualizar contacto"):
                if nombre_edit.strip() and email_edit.strip():
                    mensaje_id = None
                    if mensaje_seleccionado_edit != "Ninguno":
                        mensaje_id = mensajes.mensajes.index(mensaje_seleccionado_edit)
                    actualizado = Contacto(nombre_edit.strip(), fecha_edit.strftime("%Y-%m-%d"), email_edit.strip(), mensaje_id)
                    gestor.actualizar_contacto(idx, actualizado)
                    st.success("Contacto actualizado")
                else:
                    st.warning("⚠️ Los campos no pueden estar vacíos")

            if st.button("Eliminar contacto"):
                gestor.eliminar_contacto(idx)
                st.success("Contacto eliminado")
                st.experimental_rerun()

    elif menu == "Ver Cumpleaños":
        st.header("📅 Próximos Cumpleaños")
        if not gestor.contactos:
            st.info("No hay contactos registrados aún.")
        else:
            ordenados = sorted(gestor.contactos, key=lambda c: c.dias_para_cumple())
            for c in ordenados:
                st.write(f"📌 {c.nombre} - 📧 {c.email} - ⏳ Faltan {c.dias_para_cumple()} día(s)")

    elif menu == "Gestionar Mensajes":
        st.header("💌 Mensajes Personalizados")
        nuevo_mensaje = st.text_area("Añade un nuevo mensaje (usa {nombre} para personalizar):")
        if st.button("Guardar Mensaje"):
            if nuevo_mensaje.strip():
                mensajes.agregar_mensaje(nuevo_mensaje.strip())
                st.success("Mensaje guardado")
            else:
                st.warning("⚠️ El mensaje no puede estar vacío")

        if mensajes.mensajes:
            st.subheader("Editar o eliminar mensajes")
            opciones_msg = [f"{i+1}: {m[:40]}..." for i, m in enumerate(mensajes.mensajes)]
            seleccionado_msg = st.selectbox("Selecciona un mensaje", opciones_msg)
            idx_msg = opciones_msg.index(seleccionado_msg)
            mensaje_sel = mensajes.mensajes[idx_msg]

            texto_edit = st.text_area("Texto del mensaje", value=mensaje_sel, key="mensaje_edit")

            if st.button("Actualizar mensaje"):
                if texto_edit.strip():
                    mensajes.actualizar_mensaje(idx_msg, texto_edit.strip())
                    st.success("Mensaje actualizado")
                else:
                    st.warning("⚠️ El mensaje no puede estar vacío")

            if st.button("Eliminar mensaje"):
                mensajes.eliminar_mensaje(idx_msg)
                st.success("Mensaje eliminado")
                st.experimental_rerun()
        else:
            st.info("No hay mensajes registrados.")

if __name__ == "__main__":
    main()

Overwriting app.py


In [23]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧
up to date, audited 23 packages in 1s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠇

In [24]:
!streamlit run /content/app.py &>/content/logs.txt &

In [25]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.185.160.176
⠙your url is: https://quiet-carpets-remain.loca.lt
